In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
class CNN_low_dim(nn.Module):
    def __init__(self):
        super(CNN_low_dim,self).__init__()
        self.cnn_layers_low_dim = nn.Sequential(
                nn.Conv2d(3,8,kernel_size=3,stride=1,padding=1), # input and output channel
                nn.BatchNorm2d(8),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(8,16,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(32,8,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(8),
                nn.ReLU(inplace=True),
                )

        self.linear_layers_low_dim = nn.Sequential(
                nn.Linear(8,10)
                )

    def forward(self,x):
        x = self.cnn_layers_low_dim(x)
        #print("Shape after cnn",x.shape)
        x = x.view(x.size(0),-1)
        linear_input_shape = x.view(-1,x.shape[-1]).shape[0]
        x = self.linear_layers_low_dim(x)
        return x

class CNN_regular(nn.Module):
    def __init__(self):
        super(CNN_regular,self).__init__()

        self.cnn_layers_regular = nn.Sequential(
                nn.Conv2d(3,32,kernel_size=3,stride=1,padding=1), # input and output channel
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(256,128,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2,stride=2),
                nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1),
                nn.BatchNorm2d(64)
                )

        self.linear_layers_regular = nn.Sequential(
            nn.Linear(64,10)
        )

    def forward(self,x):
        x = self.cnn_layers_regular(x)
        x = x.view(x.size(0),-1)
        linear_input_shape = x.view(-1,x.shape[-1]).shape[0]
        x = self.linear_layers_regular(x)
        x = F.log_softmax(x,dim=1)
        return x

In [5]:
def train(epochs,data_size,reduce_dimensions):

    torch.cuda.empty_cache()
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    if reduce_dimensions:
      new_dimension = 8 # This has to be modified depending on the size of data passed
      model = CNN_low_dim().to(device)
      X_train = torch.tensor(np.load("/content/drive/MyDrive/data/X_PCA_train.npy")[:data_size,:],dtype=torch.float,requires_grad=True).reshape(-1,3,8,8).to(device)#.reshape(data_size,3,32,32).to(device)
      X_test = torch.tensor(np.load("/content/drive/MyDrive/data/X_PCA_test.npy"),dtype=torch.float).reshape(10000,3,8,8).to(device)
    else:
      model = CNN_regular().to(device)
      X_train = torch.tensor(np.load("/content/drive/MyDrive/data/X_train.npy")[:data_size,:],dtype=torch.float,requires_grad=True).reshape(-1,3,32,32).to(device)#.reshape(data_size,3,32,32).to(device)
      X_test = torch.tensor(np.load("/content/drive/MyDrive/data/X_test.npy"),dtype=torch.float).reshape(10000,3,32,32).to(device)

    train_losses = []

    y_train = torch.tensor(np.load("/content/drive/MyDrive/data/y_train.npy")[:data_size],dtype=torch.long).to(device)
    y_test = torch.tensor(np.load("/content/drive/MyDrive/data/y_test.npy"),dtype=torch.long).to(device)

    print("Training shape :",X_train.shape,y_train.shape)
    print("Testing shape :",X_test.shape,y_test.shape)

    #print("X using cuda : {}, y using cuda:{}".format(X_train.is_cuda,y_train.is_cuda))
    optimizer = optim.Adam(model.parameters(),lr=0.07)
    criterion = nn.CrossEntropyLoss().to(device)

    batch_size = 1000
    for i in range(epochs):
      iter = data_size/batch_size
      total_loss = 0
      for j in range(int(iter)):
        optimizer.zero_grad()
        output = model(X_train[j:j+batch_size])
        loss = criterion(output,y_train[j:j+batch_size])
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()
      if(i%10==0):
        print("Loss at {}th iteration is : {}".format(i,total_loss))
      train_losses.append(total_loss)
    print("The loss during training",train_losses)


    output = model(X_test)
    output = torch.argmax(output,dim=1)
    print(output.shape,output[:10])
    accuracy = sum(torch.eq(output,y_test))/y_test.shape[0]
    print("The accuracy over 10000 test images is :",accuracy)

In [ ]:
def main():
    epochs = 100
    data_size = 30000
    reduce_dimensions = False
    train(epochs,data_size,reduce_dimensions)
main()

Training shape : torch.Size([30000, 3, 32, 32]) torch.Size([30000])
Testing shape : torch.Size([10000, 3, 32, 32]) torch.Size([10000])
Loss at 0th iteration is : 62.04681169986725
Loss at 10th iteration is : 0.0016420066385762766
Loss at 20th iteration is : 0.000654751789625152
Loss at 30th iteration is : 0.000357604818418622
Loss at 40th iteration is : 0.00022432205832956242
Loss at 50th iteration is : 0.0001524061167401669
Loss at 60th iteration is : 0.00010910482455983583
Loss at 70th iteration is : 8.090056439868931e-05
Loss at 80th iteration is : 6.165956460790767e-05
Loss at 90th iteration is : 4.7959729954527575e-05
The loss during training [62.04681169986725, 42.760804533958435, 18.72180213034153, 0.703212327149231, 0.014533358582411893, 0.004578031643177383, 0.0031723601787234657, 0.0025679065147414804, 0.0021683446611859836, 0.0018725460577115882, 0.0016420066385762766, 0.001456689162296243, 0.0013039438454143237, 0.0011758665714296512, 0.0010671636227925774, 0.00097377223573

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(CNN_low_dim())

+--------------------------------+------------+
|            Modules             | Parameters |
+--------------------------------+------------+
|  cnn_layers_low_dim.0.weight   |    216     |
|   cnn_layers_low_dim.0.bias    |     8      |
|  cnn_layers_low_dim.1.weight   |     8      |
|   cnn_layers_low_dim.1.bias    |     8      |
|  cnn_layers_low_dim.4.weight   |    1152    |
|   cnn_layers_low_dim.4.bias    |     16     |
|  cnn_layers_low_dim.5.weight   |     16     |
|   cnn_layers_low_dim.5.bias    |     16     |
|  cnn_layers_low_dim.8.weight   |    4608    |
|   cnn_layers_low_dim.8.bias    |     32     |
|  cnn_layers_low_dim.9.weight   |     32     |
|   cnn_layers_low_dim.9.bias    |     32     |
|  cnn_layers_low_dim.12.weight  |    2304    |
|   cnn_layers_low_dim.12.bias   |     8      |
|  cnn_layers_low_dim.13.weight  |     8      |
|   cnn_layers_low_dim.13.bias   |     8      |
| linear_layers_low_dim.0.weight |     80     |
|  linear_layers_low_dim.0.bias  |     1

8562